In [119]:
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
import requests
import json
import re

In [120]:
# 사용자 에이전트
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:60.0) Gecko/20100101 Firefox/60.0',
    'Referer': 'https://www.musinsa.com/',
    'Accept-Language': 'en-US,en;q=0.5'
}

In [ ]:
middle_category_nums = ['001006', '001004', '001005', '001010', '001002', '001003',
                        '001001', '001011', '001013', '001008', '002022', '002001',
                        '002002', '002025', '002017', '002003', '002020', '002019',
                        '002023', '002018', '002004', '002008', '002007', '002024',
                        '002009', '002013', '002012', '002016', '002021', '002014',
                        '002006', '002015', '003002', '003007', '003008', '003004',
                        '003009', '003005', '003010', '003011', '003006', '002006',
                        '002007', '002008', '022001', '022002', '022003']

In [121]:
# df = pd.read_csv('product_numbers.csv')['num']

In [123]:
def get_response(url, headers):

    # GET 요청
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    time.sleep(0.5)

    return soup


In [ ]:
def get_items(num):
    
    url = f'https://www.musinsa.com/categories/item/{num}?d_cat_cd={num}&brand=&list_kind=small&sort=sale_high&sub_sort=1d&page=1&display_cnt=90&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure='
    
    product_links = []
    flag = 0
    page = 1
    while flag == 0:
        
        soup = get_response(url, headers)

        products = soup.select('a.img-block')
        
        for product in products:
            product_links.append(product['href'].replace('//', 'https://'))
            if len(product_links) == ITEMS_COUNT:
                flag = 1
                break

        page += 1
        url = f'https://www.musinsa.com/categories/item/{num}?d_cat_cd={num}&brand=&list_kind=small&sort=sale_high&sub_sort=1d&page={page}&display_cnt=90&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure='

    return product_links
        


In [156]:
def extract_one_review(review, product_num):
    # 댓글 단 사람
    name = review.select('p.review-profile__name')[0].text
    
    # 메타 데이터
    meta_dict = {}
    metas = review.select('li.review-evaluation--type3__item')
    for meta in metas:
        key, value = meta.text.split(' ', 1)  # 공백을 기준으로 처음 나오는 부분만 분리
        meta_dict[key] = value

    # 리뷰 내용
    content = review.select('div.review-contents__text')[0].text

    # 배지
    badge = review.select('span.review-evaluation-button--type3__count')

    # 도움돼요
    helpful = badge[0].text

    # 스타일 좋아요
    try:
        style_good = badge[1].text
        
    except:
        style_good = ''

    # 별 개수
    

    data = {
        'product_num': product_num,
        'name': name,
        'meta_data': meta_dict,
        'content': content,
        'helpful': helpful,
        'style_good': style_good
    }

    return data

In [157]:
def extract_page_review_info(soup, product_num):
    one_page_reviews = []
    reviews = soup.select('div.review-list')
    for review in reviews:
        review_dict = extract_one_review(review, product_num)
        one_page_reviews.append(review_dict)
        
    return one_page_reviews


In [161]:
def main():
    all_product_links = []
    for middle_category_num in middle_category_nums:
        product_links = get_items(middle_category_num)
        all_product_links += product_links
        print(len(all_product_links), all_product_links[-1]) 

    product_nums = [link.split('/')[-1] for link in all_product_links]

    all_reviews = []
    for product_num in product_nums:
        review_types = ['style', 'goods', 'photo']
        product_num = str(product_num)
        for review_type in review_types:
            page_num = 1

            while True:
                url = f'https://goods.musinsa.com/api/goods/v2/review/{review_type}/list?similarNo=0&sort=up_cnt_desc&selectedSimilarNo={product_num}&page={page_num}&goodsNo={product_num}&rvck=202404150551&_=1713416512533'
                soup = get_response(url, headers)
                
                if not soup.select('p.review-profile__name'):
                    break   

                one_page_reviews = extract_page_review_info(soup, product_num)
                for one_page_review in one_page_reviews:
                    all_reviews.append(one_page_review)
                
                page_num += 1


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
283
293
303
313
323
333
343
353
363
373
383
393
403
413
423
433
443
453
463
473
483
493
503
513
523
533
543
553
563
573
583
593
603
613
623
633
643
653
663
673
683
693
703
713
723
733
743
753
763
773
783
793
803
813
823
833
843
853
863
873
883
893
903
913
923
933
943
953
963
973
983
993
1003
1013
1023
1033
1043
1053
1063
1073
1083
1093
1103
1113
1123
1133
1143
1153
1163
1173
1183
1193
1203
1213
1223
1233
1243
1253
1263
1273
1283
1293
1303
1313
1323
1333
1343
1353
1363
1373
1383
1393
1403
1413
1423
1433
1443
1453
1461
1471
1481
1491
1501
1511
1521
1531
1541
1551
1561
1571
1581
1591
1601
1611
1621
1631
1641
1651
1661
1671
1681
1691
1701
1711
1721
1731
1741
1751
1761
1771
1781
1791
1801
1805
1815
1825
1835
1845
1855
1865
1875
1885
1895
1905
1915
1925
1935
1945
1955
1965
1975
1985
1995
2005
2015
2025
2035
2045
2055
2065
2075
2085
2095
2105
2115
2125
2135
2145
2155
2165
2175
2185
2195
2205

In [166]:
pd.DataFrame(all_reviews)['meta_data']

0       {'사이즈': '보통이에요', '밝기': '어두워요', '색감': '선명해요', '...
1       {'사이즈': '커요', '밝기': '어두워요', '색감': '보통이에요', '두께...
2       {'사이즈': '보통이에요', '밝기': '보통이에요', '색감': '보통이에요',...
3       {'사이즈': '보통이에요', '밝기': '보통이에요', '색감': '보통이에요',...
4       {'사이즈': '보통이에요', '밝기': '보통이에요', '색감': '보통이에요',...
                              ...                        
7038    {'사이즈': '보통이에요', '밝기': '보통이에요', '색감': '보통이에요',...
7039    {'사이즈': '보통이에요', '밝기': '보통이에요', '색감': '보통이에요',...
7040    {'사이즈': '보통이에요', '밝기': '보통이에요', '색감': '보통이에요',...
7041    {'사이즈': '보통이에요', '밝기': '밝아요', '색감': '선명해요', '두...
7042    {'사이즈': '보통이에요', '밝기': '보통이에요', '색감': '보통이에요',...
Name: meta_data, Length: 7043, dtype: object

In [ ]:
if __name__ == '__main__':
    main()